In [84]:
# coding: utf-8
import pandas as pd
from pathlib import Path
import json
root = Path('.')
root = root.absolute()
test_cases_root = root / "testcases"
results_root = root / "fmcad_results_2023-05-06_10-28-32/"

In [85]:
print("root: ", root)
print("test_cases_root: ", test_cases_root)
print("results_root: ", results_root)
# cwd
print("cwd: ", Path.cwd())

root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether
test_cases_root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether/testcases
results_root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether/fmcad_results_2023-05-06_10-28-32
cwd:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether


In [91]:
test_cases = [y for x in test_cases_root.iterdir() for y in x.iterdir() if y.suffix == ".th"]
rel_cases = [x.relative_to(test_cases_root) for x in test_cases]
configs = [x.name for x in results_root.iterdir()]
jsons = {n: pd.read_csv(results_root /  n / "stats.csv") for n in configs}
# Rename columns such that Unnamed: 0 is the test suite nameL
jsons = {n: j.rename(columns={"Unnamed: 0": "test_suite"}) for n, j in jsons.items()}
# Assert test suite column of first configurations contains at least one case of clam
assert any(jsons[configs[0]]["test_suite"].str.contains("clam"))

In [93]:
from numpy import NaN


colored_stats = {}
# Calculate relative_path for each test case
for n, j in jsons.items():
    j["relative_path"] = j.apply(lambda x: (Path(x["test_suite"]) / x['file_name'].replace('stats.json', 'th')), axis=1)
    j["total_enodes"] = NaN
    # Find all existing colored_stats.json files by iterating over j['relative_path']
    def collect_total(row):
        x = row['relative_path']
        # Find all colored_stats.json files
        cs_path = results_root / n / x.with_suffix(".colored_stats.json")
        if cs_path.exists():
            with cs_path.open() as f:
                cs = json.load(f)
                colored_stats[(n, x)] = cs
            filtered = [x[1] for x in cs]
            for stats in filtered:
                stats['total_enodes'] = stats['black_size']
                if 'colors_sizes' in stats:
                    stats['total_enodes'] += sum(stats['colors_sizes'].values())
                if 'split_sizes' in stats:
                    stats['total_enodes'] += sum(stats['split_sizes'])
            # take max of total_enodes from filtered and add it to j
            return max([x['total_enodes'] for x in filtered])
        else:
            return Nan
    j['total_enodes'] = j.apply(collect_total, axis=1)

In [94]:
# Start adding missing files by first adding an error column
for j in jsons.values():
    j["error"] = None

# Now add the missing files. For each missing case find the corresponding error file
for n, j in jsons.items():
    # Find missing cases by relative path to rel_cases
    in_df = j['relative_path'].unique()
    missing_cases = [r for r in rel_cases if r not in in_df]
    for fn in missing_cases:
        errors = (results_root / n / fn.with_suffix('.err')).read_text()
        j.loc[len(j)] = {"relative_path": fn, "error": errors}
        

In [95]:
for c in configs:
    if not ('keep' in c or 'colored' in c):
        continue
    # Filter errors by x is not None and ('Timeout' in x or 'memory' in x)
    temp = jsons[c]['error'].apply(lambda x: x is not None and ('Timeout' in x or 'memory' in x))
    print(c)
    print(jsons[c][temp]['error'].value_counts())


case_split_proof_split_3_split_clone_keep_splits_stats
error
Timeout Exception                                 25
memory allocation of 4194304 bytes failed\n        1
memory allocation of 1137600 bytes failed\n        1
memory allocation of 8 bytes failed\n              1
memory allocation of 11003664 bytes failed\n       1
memory allocation of 240 bytes failed\n            1
memory allocation of 1845493760 bytes failed\n     1
Name: count, dtype: int64
case_split_proof_split_3_split_colored_stats
error
Timeout Exception                                  21
memory allocation of 60129542144 bytes failed\n     1
memory allocation of 30064771072 bytes failed\n     1
Name: count, dtype: int64
case_split_proof_split_4_split_colored_stats
error
Timeout Exception                                  39
memory allocation of 60129542144 bytes failed\n     1
memory allocation of 30064771072 bytes failed\n     1
Name: count, dtype: int64
case_split_proof_split_4_split_clone_keep_splits_stats
error
Tim

In [ ]:
print(jsons['case_split_proof_split_4_split_colored_stats']['error'].value_counts())

error
Timeout Exception                                                                                                                                                                                                                                                                                                                      39
14:58:25 [ERROR] Unrecognized token `(` found at 2707:2708\nExpected one of "then"\nthread 'main' panicked at 'Please implement error handling', src/thesy/semantics.rs:123:17\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n                                                           18
thread 'main' panicked at 'called `Result::unwrap()` on an `Err` value: "Found a list in the head position: [String(\"x\"), String(\"Nat\")]"', src/bin/expl_experiment.rs:125:28\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n                                                                     13
14:58:25 [ERROR] 

In [96]:
j = list(jsons.values())[0]
print(j.columns)
j.describe()

Index(['test_suite', 'Unnamed: 1', 'time', 'stop_reasons', 'success',
       'lemma_count', 'proofs_later_filtered', 'case_split_root_count',
       'case_split_had_vacuity', 'total_allocated', 'max_allocated',
       'file_name', 'relative_path', 'total_enodes', 'error'],
      dtype='object')


,Unnamed: 1,time,lemma_count,proofs_later_filtered,case_split_root_count,total_allocated,max_allocated,total_enodes
count,107.00000,107.000000,107.0,107.0,107.0,1.070000e+02,1.070000e+02,107.000000
mean,30.17757,2.043883,0.0,0.0,0.0,1.785569e+09,8.232889e+06,409.196262
std,23.86788,18.744478,0.0,0.0,0.0,1.591609e+10,6.549279e+07,1228.321553
min,0.00000,0.002075,0.0,0.0,0.0,5.133249e+06,4.280190e+05,41.000000
25%,8.50000,0.003539,0.0,0.0,0.0,5.895558e+06,4.349290e+05,97.500000
50%,24.00000,0.004543,0.0,0.0,0.0,6.271916e+06,4.441940e+05,135.000000
75%,50.50000,0.013615,0.0,0.0,0.0,1.673392e+07,5.585015e+05,183.500000
max,77.00000,193.843561,0.0,0.0,0.0,1.644304e+11,6.761496e+08,10770.000000


In [100]:
# Write our all new csv files as updated_stats.csv
for n, j in jsons.items():
    j.to_csv(results_root / (n[23:] + "updated_stats.csv"))
